In [2]:
import os
import pandas as pd
import json
import time

In [9]:
df = pd.DataFrame()
for dataset in os.listdir("./../models"):
    if os.path.isdir(dataset) and not dataset.startswith("."):
        for task in os.listdir(dataset):
            if os.path.isdir(os.path.join(dataset, task)) and not task.startswith("."):
                with open(os.path.join(dataset, "dataset.json"), 'r') as f:
                    dataset_info = json.load(f)
                    dataset_info = dict([(key, dataset_info[0][key]) for key in 
                                         ['number_of_features',
                                          'number_of_instances',
                                          'number_of_missing_values',
                                          'number_of_instances_with_missing_values']])
                    if not task.startswith("classification"):
                        continue
                    for model in os.listdir(os.path.join(dataset, task)):
                        try:
                            if os.path.isdir(os.path.join(dataset, task, model)):
                                with open(os.path.join(dataset, task, model, "audit.json")) as fa:
                                    audit_info = json.load(fa)
                                    audit_info = dict([(metrics, audit_info[0]['performance'][metrics]) 
                                                       for metrics in ['acc', 
                                                                       'auc',
                                                                       'specificity',
                                                                       'recall', 
                                                                       'precision',
                                                                       'f1']])
                                with open(os.path.join(dataset, task, model, "model.json")) as fm:
                                    model_info = json.load(fm)
                                    model_info = dict([(key, model_info[0][key]) 
                                                      for key in ['id',
                                                                  'library',
                                                                  'model_name']])
                                model = dataset_info
                                model.update(audit_info)
                                model.update(model_info)
                                model = pd.DataFrame.from_dict(model, orient='index').T
                                print(model)
                                df = df.append(model)
                        except:
                            continue